In [1]:
import json
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np

/home/suigpt_rag/miniconda3/envs/new_CG/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from openai import OpenAI
API_KEY = "OPENAI_API_KEY_PLACEHOLDER"
client = OpenAI(api_key=API_KEY)

In [10]:
# r = client.embeddings.create(
#             input=["test", "test2", "test3"],            
#             model="text-embedding-3-large"
# )
# print(r.data[0].embedding)

[-0.02476464956998825, -0.008327321149408817, -0.009756051003932953, 0.04675506427884102, 0.006940205115824938, 0.026669621467590332, -0.012576520442962646, 0.07220402359962463, 0.010939723812043667, 0.042501240968704224, -0.01461095828562975, 0.02347000688314438, -0.0025222396943718195, -0.022212354466319084, 0.008503022603690624, 0.007989789359271526, 0.005483733024448156, 0.01294641848653555, -0.026688117533922195, 0.0022112946026027203, 0.012206623330712318, -0.02570788748562336, 0.004230704624205828, 0.01925317384302616, -0.03279142826795578, 0.021065672859549522, 0.03835838660597801, 0.007111282553523779, -0.014638700522482395, -0.015221289359033108, 0.008327321149408817, 0.053635161370038986, 0.002487561898306012, 0.0014645635383203626, 0.0007028055842965841, -0.0013489705743268132, 0.03745213896036148, -0.012317592278122902, 0.023303553462028503, 0.0031649370212107897, 0.009756051003932953, -0.029573319479823112, -0.0023789044935256243, 0.017098519951105118, -0.0399489477276802

In [5]:
# 1. import testing Question dataset [Q]
with open("../math_qa_all_v1.json", "r") as qf:
    data = json.load(qf)
    page_content = [q['content'] for q in data]
    page_number = [q['page'] for q in data]
    questions: list = [q['Q'] for q in data]
    answers: list = [q['A'] for q in data]

In [25]:
# 4.7s
response = client.embeddings.create(
            input=page_content,            
            model="text-embedding-3-large"
)

page_embeddings = [d.embedding for d in response.data]

In [26]:
# 2.7s
response = client.embeddings.create(
            input=questions,            
            model="text-embedding-3-large"
)

questions_embeddings = [d.embedding for d in response.data]

In [27]:
for i in range(len(data)):
    data[i]['page_embedding'] = page_embeddings[i]
    data[i]['question_embedding'] = questions_embeddings[i]

In [28]:
# question_retrived_page_rank = []
for i in range(len(data)):
    similarity = [float(cos_sim(data[i]['question_embedding'], page_embed)) for page_embed in page_embeddings]
    question_retrived_page_rank = [{k: v} for k, v in zip(page_number, similarity)]
    data[i]['question_retrived_page_rank'] = question_retrived_page_rank

In [29]:
for i in range(len(data)):
    data[i]['question_retrived_page_rank'] = sorted(data[i]['question_retrived_page_rank'], key = lambda d: list(d.values())[0], reverse=True)

In [30]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0

page_distances = [] # the distances of the correct page and retrieved page
scores = [] # the score of the corresponding page and question
correct_rank = [] # the index of the correct page in the retrieved results.

for i in range(len(data)):
    page_number = data[i]['page']
    retrived_page_rank = [list(data[i]['question_retrived_page_rank'][d].keys())[0] for d in range(len(data[i]['question_retrived_page_rank']))]
    correct_page_rank = retrived_page_rank.index(page_number)
    correct_rank.append(correct_page_rank)
    page_distances.append(abs(page_number - retrived_page_rank[0]))
    scores.append(cos_sim(data[i]['question_embedding'], data[i]['page_embedding']))
    
    if page_number in retrived_page_rank[:1]: top1 += 1
    if page_number in retrived_page_rank[:3]: top3 += 1
    if page_number in retrived_page_rank[:5]: top5 += 1
    if page_number in retrived_page_rank[:10]: top10 += 1
    if page_number in retrived_page_rank[:20]: top20 += 1
    if page_number in retrived_page_rank[:50]: top50 += 1
    if page_number in retrived_page_rank[:100]: top100 += 1   

In [31]:
print(top1)
print(top3)
print(top5)
print(top10)
print(top20)
print(top50)
print(top100)


262
387
426
445
468
473
477


In [32]:
print("Avg score: " + str(float(sum(scores) / len(scores))))
print("Avg rank: " + str(sum(correct_rank) / len(correct_rank)))
print("Avg page distance: " + str(sum(page_distances)/len(page_distances)))

Avg score: 0.5780389308929443
Avg rank: 2.3815513626834384
Avg page distance: 16.9811320754717


In [33]:
import pickle

with open('openAI.pkl', 'wb') as f:
    pickle.dump(data, f)